In [1]:
# olmocr python 가상환경 설정

In [2]:
# !pip install -q nest_asyncio

In [3]:
import nest_asyncio
#nest_asyncio는 Jupyter Notebook(ipynb)에서 비동기(async/await) 코드를 실행하기 위해 필요
nest_asyncio.apply()

## OLMOCR 설치

In [4]:
# RestFul서비스를 구축하고 테스트하기 위해 필요
# !pip install -Uq olmocr openai ipywidgets 

In [5]:
# !pip install transformers==4.49.0

In [6]:
# !git clone https://github.com/allenai/olmocr.git
# %cd olmocr
# !pip install -e .

## PDF 테스트 파일 설정

In [7]:
import asyncio
import json
from pathlib import Path
from tqdm import tqdm
import os

from pypdf import PdfReader
from olmocr.pipeline import build_page_query, build_dolma_document, PageResult
from olmocr.prompts import PageResponse

from olmocr.version import VERSION
from glob import glob

from openai import OpenAI
client = OpenAI(base_url="http://localhost:8000/v1", api_key="token-abc123", timeout=120)

async def process_page(filename, page_num):
    query = await build_page_query(filename,
                            page=page_num,
                            target_longest_image_dim=1024,
                            target_anchor_text_len=6000)
    
    # query['model'] = 'Qwen2.5-VL-7B-Instruct'
    query['model'] = 'olmOCR-7B-0225-preview'

    # print(query)
    response = client.chat.completions.create(**query)

    content = response.choices[0].message.content

    page_response = PageResponse(
        primary_language = "ko",
        is_rotation_valid = False,
        rotation_correction = 0,
        is_table = False,
        is_diagram = False,
        natural_text = content
    )

    return PageResult(
        filename,
        page_num,
        page_response,
        input_tokens=response.usage.prompt_tokens,
        output_tokens=response.usage.completion_tokens,
        is_fallback=False
    )

In [8]:
# async def get_content(file_path):
#     reader = PdfReader(file_path)
#     file_name = Path(file_path).stem
#     num_pages = reader.get_num_pages()
#     results = []
    
#     for page_num in trange(1, num_pages + 1):
#         result = await process_page(file_path, page_num)
        
#         results.append(result)
        
#         print(result.response)
        
#         print("--------------------------------")
#         # 만약 result.response가 JSON 문자열이라면
#         if isinstance(result.response.natural_text, str):
#             try:
#                 response_dict = json.loads(result.response.natural_text)
#                 print(f"Processed: {response_dict['natural_text']}")
#             except json.JSONDecodeError:
#                 print(f"Processed: {result.response.natural_text}")
#         else:
#             # PageResponse 객체라면
#             print(f"Processed: {result.response.natural_text}")
            
#     print("--- Get PDF Results ---")
#     text_parts = []
#     for result in results:
#         if isinstance(result.response.natural_text, str):
#             try:
#                 response_dict = json.loads(result.response.natural_text)
#                 text_parts.append(response_dict['natural_text'])
#             except json.JSONDecodeError:
#                 # JSON 파싱에 실패하면 원본 텍스트 추가
#                 text_parts.append(result.response.natural_text)
#         else:
#             text_parts.append(result.response.natural_text)
    
#     text = "\n".join(text_parts)
#     print(text)

#     # 텍스트를 마크다운 파일로 저장
#     output_folder = Path("./results")
#     output_folder.mkdir(exist_ok=True, parents=True)
    
#     md_path = output_folder.joinpath(file_name).with_suffix(".md")
#     full_path = md_path.resolve()
#     print(f"마크다운 저장 경로: {full_path}")
#     with open(full_path, "w", encoding="utf-8") as file:
#         file.write(text)

#     # build_dolma_document 부분 주석 처리
#     # print("--- Build the dolma results ---")
#     # doc = build_dolma_document(file_path, results)
#     # pdf_doc = json.dumps(doc, ensure_ascii=False)
#     # print(pdf_doc)
#     # dolma_path = output_folder.joinpath(file_name).with_suffix(".results.json")
#     # full_path = dolma_path.resolve()
#     # print(full_path)
#     # with open(full_path, "w", encoding="utf-8") as file:
#     #     file.write(pdf_doc + "\n")

In [9]:
async def get_content(file_path, single_page=False, page_num=None):
    reader = PdfReader(file_path)
    file_name = Path(file_path).stem
    num_pages = reader.get_num_pages()
    results = []
    
    # 파일명에 대한 폴더 생성
    output_folder = Path("./results")
    output_folder.mkdir(exist_ok=True, parents=True)
    
    pdf_folder = output_folder.joinpath(file_name)
    
    # 폴더가 이미 존재하는지 확인
    if pdf_folder.exists() and len(os.listdir(pdf_folder)) == num_pages and not single_page:
        print(f"폴더 '{pdf_folder}'가 이미 존재합니다. 처리를 건너뜁니다.")
        return  # 함수 종료
    
    pdf_folder.mkdir(exist_ok=True, parents=True)
    
    page_num = 1 if page_num is None else page_num
    pbar = tqdm(total=num_pages if not single_page else 1)
    
    while page_num <= num_pages:
        result = await process_page(file_path, page_num)

        print("--------------------------------")
        
        # 페이지 텍스트 추출
        page_text = None
        if isinstance(result.response.natural_text, str):
            try:
                response_dict = json.loads(result.response.natural_text)
                page_text = response_dict.get('natural_text')
                # print(f"Processed: {page_text}")
            except json.JSONDecodeError:
                print(f"Error: JSONDecodeError")                
        else:
            page_text = result.response.natural_text
            # print(f"Processed: {page_text}")
        
        # None이거나 빈 문자열이면 재시도
        if page_text is None or page_text == "":
            print(f"페이지 {page_num}의 텍스트가 없습니다. 재시도합니다.")
            # 잠시 대기 후 재시도
            await asyncio.sleep(1)
            continue  # 같은 페이지 다시 시도
        
        # 성공적으로 텍스트를 얻었으면 결과 저장 및 다음 페이지로
        results.append(result)
        
        # 페이지별 마크다운 파일 저장
        page_md_path = pdf_folder.joinpath(f"page_{page_num}.md")
        with open(page_md_path, "w", encoding="utf-8") as file:
            file.write(page_text)
        
        if single_page:
            break
        
        # 다음 페이지로 이동
        page_num += 1
        pbar.update(1)
    
    print("---End---")
    pbar.close()

In [10]:
def main():
    file_dir = "data/건설안전지침/"
        
    files = glob(file_dir + "*.pdf")
    
    for file in files:
        print(f"Processing {Path(file).stem}")
        # Run the async event loop
        asyncio.run(get_content(file))

### 특정 문서 / 페이지 디버깅을 위한 함수

In [11]:
# def main():
#     files = ["data/건설안전지침/재사용 가설기자재 성능기준에 관한 지침.pdf"]
    
#     for file in files:
#         print(f"Processing {Path(file).stem}")
#         # Run the async event loop
        
#         for page_num in [23, 25, 28, 30, 33, 37, 42, 43, 44, 45, 48, 49]:
#             asyncio.run(get_content(file, single_page=True, page_num=page_num))
#         # asyncio.run(get_content(file, single_page=True, page_num=14))

In [12]:
if __name__ == "__main__":
    main()

INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
 58%|█████▊    | 14/24 [02:20<01:57, 11.73s/it]

--------------------------------
Processed: 이 발생되지 않도록 하여야 한다.

6.2 강관 추진 및 굴착

6.2.1 강관 추진

(1) 강관의 추진순서는 강관 주변의 토사의 종류 및 상태, 작업 여건 등을 고려하여 결정하여야 한다.

(2) 강관은 추진시 선형 조정이 용이하고 마찰이 작은 길이로 제작하여야 한다.

(3) 상부 토압에 견딜 수 있는 충분한 두께의 강관을 사용하여야 한다.

(4) 처음으로 추진되는 강관은 정확하게 압입되어야 후속 추진되는 강관이 정확하게 추진되므로 시공관리를 철저히 하여야 한다.

(5) 강관추진기에는 추진 시 추진력이 감자가 상승하는 것을 방지하기 위한 압력 리미트(Limit) 스위치를 설치하여야 한다.

(6) 추진은 지반의 토질 및 막장 상태를 고려하여 신중히 하여야 하고, 추진 중에 추진력이 감자가 상승하는 경우에는 추진을 중지하고 그 원인을 조사하여 안전을 확인한 뒤에 추진하여야 한다.

(7) 강관을 미리 용접할 경우 오차를 수정하기가 힘들 뿐 아니라 추진 시 강관이 주변 토사와 마찰력이 과도하게 발생되기 때문에 강관의 용접은 강관추진이 완료된 후에 실시하여야 한다.

(8) 강관과 강관의 용접결합은 작업조건이 매우 불량한 협소한 장소에서 작업이 이루어지므로 최소 2인 이상이 상호 교대로 작업을 실시하고 다음 사항을 준수하여야 한다.

(가) 가연물 제거와 화재예방 조치
(나) 관내 용접작업 시 환기설비 설치
(다) 상하 동시작업 시 사전 동의 후 작업
(라) 전선 및 호스 정리정돈, 안전한 호스 연결기구 사용


INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
 62%|██████▎   | 15/24 [02:30<01:41, 11.24s/it]

--------------------------------
Processed: (마) 추진관 상부 용접작업을 위한 멸어짐 방지조치
(바) 용접기 감전재해 예방조치

(9) 강관추진이 끝나고 용접이 완료된 후에는 혹시 발생할 수 있는 굴진 시의 여굴부위에 시멘트 밀크 그라우팅을 실시하여 침하를 예방하여야 한다.

(10) 강관추진 시 선단에 장애물이 있는 경우에는 관 추진 부분의 장애물을 제거한 후 즉시 강관추진을 하고 주위를 그라우팅을 하여 장애물 제거로 인한 여굴이 발생하지 않도록 주의하여야 한다.

<그림 7> 강관 압입 전경

6.2.2 강관내부 굴착

(1) 지반조건 등을 고려하여 인력굴착 및 기계굴착 방법을 결정하여야 한다.

(2) 강관내부 굴착 시 지반 및 지하수 상태를 항상 확인하여야 하며 굴착에 따른 지하수의 유출 등으로 박장에서의 무너짐 및 묻힘 등의 우려가 있을 경우에는 차수 그라우팅을 실시한 후 굴착하여야 하고 항상 안전대책을 수립하여야 한다.

(3) 인력굴착 및 기계굴착을 병행할 경우 각각의 작업범위와 작업방향을 명확히 하고 기계의 작업반경내에 근로자가 출입하지 않도록 방호설비를 하거나 작업지휘자를 반드시 배치하여야 한다.


INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
 67%|██████▋   | 16/24 [02:41<01:30, 11.33s/it]

...

--------------------------------
Processed: (라) 발판 높이는 24cm 이하이어야 하고, 일정한 간격으로 부착 고정된 구조 이어야 한다.

(2) 조립형 가설계단
조립형 가설계단은 그림 4와 같이 발판을 단관비계의 지지대에 크램프를 사용하여 작업장의 환경에 적합하게 조립한 것으로 다음 각 호의 규정에 적합하여야 하며 사용에 지장이 없어야 한다.

(가) 가설계단의 경사각은 30°~60°를 유지하여 설치하도록 하여야 한다.

(나) 발판 너비는 23cm 이상으로 하고, 또한 그 바닥면은 미끄럼 방지 기능을 가지고 있어야 한다.

(다) 발판 높이는 23cm 이하이어야 하고, 일정한 간격으로 설치하여야 한다.

(라) 단관비계와 크램프는 가설기자재 안전인증에 합격한 제품을 사용하여야한다.

8. 가설계단 설치작업 중 안전 준수사항

(1) 당해 작업의 관리감독자는 근로자에게 작업장소의 상황과 작업순서 등을 가능한 그림 등을 사용하여 정확하게 지시하여야 하며, 작업 중 예상되는 위험요인과 안전수칙을 충분히 설명하고 그 이행여부를 감독하여야 한다.

(2) 근로자는 당해 작업에 적합한 개인보호구를 착용하여야 하며, 안전한 작업을 수행하도록 안전수칙을 준수하여야 한다.


INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
 91%|█████████ | 10/11 [01:58<00:12, 12.93s/it]

--------------------------------
Processed: (3) 작업의 안전한 진행을 위하여 근로자의 경험 및 숙련여부를 고려하여 작업에 배치되도록 하여야 한다.

(4) 작업구역 내에는 관계 근로자 외의 자의 출입을 금지시켜야 한다.

(5) 출입금지 장소에는 울타리, 로프, 표지 등을 설치하여 작업장소에 접근할 수 없도록 하고, 필요에 따라 감시자를 배치하여야 한다.

(6) 사용하는 기기, 공구 등은 정상적으로 작동할 수 있도록 작업전에 점검하고 불량공구는 사용을 금지하거나 정비한 다음에 사용하여야 한다.

(7) 가설계단의 설치를 완료하면 설치상태를 점검한 후에 사용하도록 하여야 한다.

9. 가설계단의 유지관리

가설계단을 설치, 사용 및 유지관리를 위해 정기적으로 점검하고, 불량 혹은 이상이 발견되었을 경우에는 즉시 보수하도록 한다.

(1) 설치 작업 중 점검

(가) 재료가 규격에 적합한지 아닌지를 확인하여야 한다.

(나) 재료에 녹, 변형 또는 손상 등에 의한 결함이 없는 지를 확인하여야 한다.

(다) 발판이 설치기준에 따라 적합하게 되었는지를 확인하여야 한다.

(2) 사용 및 유지관리 점검

(가) 발판과 지지대의 접속이나 연결부의 이상 유무를 확인하여야 한다.

(나) 발판의 변형 부식 또는 손상유무를 확인하여야 한다.

(다) 발판위에 떨어져있는 자재나 공구 등의 유무를 확인하여야 한다.

(라) 계단 주변의 통로 확보 및 정리정돈 상태를 확인하여야 한다.


INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
100%|██████████| 11/11 [02:18<00:00, 12.60s/it]

--------------------------------
Processed: 부록: 참고자료

1. 비계의 조립형 계단 설치기준 (출처: BS EN 12811-1 : 2003)

*명칭: 1. 발판
2. 계단참
*기준(단위: cm)
1) 발판높이: 17.5 ≤ u ≤ 22.5
2) 겹침: -2.5 ≤ a ≤ 5.5
3) 발판너비

| 구분 | A형       | B형       |
|------|-----------|-----------|
| s    | 12.5 ≤ s < 16.5 | 16.5 ≤ s  |
| g    | 15.0 ≤ g < 17.5 | 17.5 ≤ g  |

2. 기계 · 탱크 · 설비의 내·외부 계단이나 마루 · 난간 · 피트에 연결된 고정 계단의 설치기준 (출처: OSHA 1910.24(e) Angle of stairway rise)

| 경사각 | 발판 높이(cm) | 발판 너비(cm) |
|--------|---------------|---------------|
| 30°35' | 16.51         | 27.94         |
| 32°08' | 17.15         | 27.31         |
| 33°41' | 17.78         | 26.67         |
| 35°16' | 18.42         | 26.04         |
| 36°52' | 19.05         | 25.40         |
| 38°29' | 19.69         | 24.77         |
| 40°08' | 20.32         | 24.13         |
| 41°44' | 20.96         | 23.50         |
| 43°22' | 20.59         | 22.86         |
| 45°00' | 22.23         | 22.23         |
| 46°38' | 22.86         | 21.59         |
| 48°16' | 23.50     